In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
rejected_sample = pd.read_csv('/Users/eduardoangeli/Desktop/Capstone/Dataset/rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv')
accepted_sample = pd.read_csv('/Users/eduardoangeli/Desktop/Capstone/Dataset/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv')
rejected_sample['target'] = 0
accepted_sample['target'] = 1
merged_data = pd.concat([rejected_sample, accepted_sample], axis=0, ignore_index=True)


/var/folders/r6/9sr46wnj2jnbq3yrgp4bf0c80000gn/T/ipykernel_1707/2645925258.py:2: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_sample = pd.read_csv('/Users/eduardoangeli/Desktop/Capstone/Dataset/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv')


In [3]:
# Feature engineering and preprocessing
bins = [-float('inf'), merged_data['loan_amnt'].quantile(0.33), merged_data['loan_amnt'].quantile(0.66), float('inf')]
labels = [0, 1, 2]
merged_data['loan_amnt_binned'] = pd.cut(merged_data['loan_amnt'], bins=bins, labels=labels)
features = ['loan_amnt_binned', 'dti', 'addr_state', 'emp_length']
X = merged_data[features]
y = merged_data['target']

# Identify columns in X with NaN values
nan_columns = X.columns[X.isnull().any()]

# Handle NaN values comprehensively
for column in nan_columns:
    if X[column].dtype == 'object' or isinstance(X[column].dtype, pd.CategoricalDtype):
        X[column].fillna(X[column].mode()[0], inplace=True)  # fill with the most frequent category
    else:
        X[column].fillna(X[column].median(), inplace=True)

# Convert categorical variables to numerical
label_encoders = {}
for col in ['addr_state', 'emp_length']:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Split data and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
classifier = LogisticRegression(random_state=42)
classifier.fit(X_train, y_train)


/var/folders/r6/9sr46wnj2jnbq3yrgp4bf0c80000gn/T/ipykernel_1707/2844896806.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column].fillna(X[column].mode()[0], inplace=True)  # fill with the most frequent category
/var/folders/r6/9sr46wnj2jnbq3yrgp4bf0c80000gn/T/ipykernel_1707/2844896806.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column].fillna(X[column].median(), inplace=True)
/var/folders/r6/9sr46wnj2jnbq3yrgp4bf0c80000gn/T/ipykernel_1707/2844896806.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

LogisticRegression(random_state=42)

In [4]:
def predict_approval(loan_amnt, dti, addr_state, emp_length):
    # Convert loan amount to binned value
    if loan_amnt <= merged_data['loan_amnt'].quantile(0.33):
        loan_amnt_binned = 0
    elif loan_amnt <= merged_data['loan_amnt'].quantile(0.66):
        loan_amnt_binned = 1
    else:
        loan_amnt_binned = 2
    
    # Convert categorical variables using label encoders
    addr_state_encoded = label_encoders['addr_state'].transform([addr_state])[0]
    emp_length_encoded = label_encoders['emp_length'].transform([emp_length])[0]

    # Create feature vector and standardize
    feature_vector = scaler.transform([[loan_amnt_binned, dti, addr_state_encoded, emp_length_encoded]])

    # Predict using the trained classifier
    prediction = classifier.predict(feature_vector)
    
    return "Approved" if prediction[0] == 1 else "Rejected"

In [5]:
# Example usage
result = predict_approval(10000, 0.2, "CA", "10+ years")
print(result)


Rejected


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
